In [0]:
# 데이터 체크 : movieidx \t review \t score

rawtext= 'merged_comments.txt'
with open(rawtext, encoding= 'utf8')as f:
    for _ in range(10):
        print(f.readline().strip())

45290	크리스토퍼 놀란 에게 우리는 놀란 다	10
45290	인셉션 정말 흥미진진하게 봤었고 크리스토퍼 놀란 감독님 신작 인터스텔라도 이번주 일요일에 보러갑니다 완전 기대중	10
45290	놀란이면 무조건 봐야 된다 왜냐하면 모든 작품을 다 히트 쳤으니깐	10
45290	나는 감탄할 준비가 되어있다	10
45290	얘들아 오늘나오는거지 밤에 ㅋㅋ 오늘 보러가야겟다	10
45290	이제 죽어도 여한이 없다	10
45290	기대감에 잠도 안온다	10
45290	나랑 같이 봐 줄까 ㅎ	10
45290	우선 명감독들이 모였고 미국에서도 극찬을 받았더군	10
45290	드디어 내일 꼭본다	10


In [0]:
# 데이터 로드

def load_reviews(fname, n_limit=-1): 
    with open(fname, encoding='utf-8') as f:
        idxs = []
        scores = []
        texts = []
        
        for i, doc in enumerate(f):
            if n_limit > 0 and i >= n_limit:
                break
                
            try:
                idx, text, score = doc.strip().split('\t')
                idxs.append(idx)
                texts.append(text.strip())
                scores.append(int(score))
                
            except Exception as e:
                print(e)
                continue
                
    return idxs, texts, scores

idxs, texts, scores = load_reviews(rawtext)
print(len(idxs))

294493


In [0]:
from collections import Counter

word_count = Counter( [word for text in texts for word in text.strip().split()] )

# 빈도수 상위 20개 단어 출력 / .itmes() method


# 1~n 번 이상 출현한 단어들의 수 출력



[('영화', 40000),
 ('정말', 18266),
 ('진짜', 14207),
 ('너무', 13397),
 ('이', 7893),
 ('영화를', 7006),
 ('그냥', 6844),
 ('더', 6560),
 ('최고의', 6394),
 ('보고', 5899),
 ('좀', 5725),
 ('수', 5671),
 ('영화가', 5588),
 ('최고', 5347),
 ('영화는', 5308),
 ('잘', 5039),
 ('꼭', 4944),
 ('ㅋㅋ', 4865),
 ('본', 4655),
 ('다', 4609)]

limit = 1, the number of words : 400649
limit = 2, the number of words : 100035
limit = 3, the number of words : 61982
limit = 4, the number of words : 46182
limit = 5, the number of words : 37222
limit = 6, the number of words : 31295
limit = 7, the number of words : 27206


In [0]:
# 인풋 텍스트에서 빈도 n 이상인 단어들만을 남기는 함수
def my_word_tokenizer(sentence_in_string):
    
    return sentence_tokenized # list

print(texts[2])
print(my_word_tokenizer(texts[2]))


37222
놀란이면 무조건 봐야 된다 왜냐하면 모든 작품을 다 히트 쳤으니깐
['놀란이면', '무조건', '봐야', '된다', '왜냐하면', '모든', '작품을', '다', '히트']


In [0]:
#영화 평점 별 리뷰 수

for score, score_freq in sorted(Counter(scores).items()):
    print('score = %d: (%d, %.3f perc)' % (score, score_freq, 100*score_freq/len(scores)))

score = 1: (56122, 19.057 perc)
score = 2: (4725, 1.604 perc)
score = 3: (4547, 1.544 perc)
score = 4: (4062, 1.379 perc)
score = 5: (7697, 2.614 perc)
score = 6: (7588, 2.577 perc)
score = 7: (11338, 3.850 perc)
score = 8: (20311, 6.897 perc)
score = 9: (25528, 8.668 perc)
score = 10: (152575, 51.809 perc)


In [0]:
# 1~3 점은 -1
# 9,10점은 1
# 나머지는 사용하지 않음

train_texts = []
train_label = []

for review, score in zip(texts, scores):
    if 3 < score < 9:
        continue
    tokenized_review = my_word_tokenizer(review)
    if not tokenized_review:
        continue
  
    train_texts.append(tokenized_review)
    train_label.append(1 if score >= 9 else -1)

print('train data: %d --> %d' % (len(texts), len(train_texts)))
for label, label_freq in Counter(train_label).items():
    print('label = %d: (%d, %.3f perc)' % (label, label_freq, 100*label_freq/len(train_label)))

train data: 294493 --> 226572
label = 1: (166225, 73.365 perc)
label = -1: (60347, 26.635 perc)


In [0]:
# 단어의 빈도수로 벡터화

from sklearn.feature_extraction.text import CountVectorizer


vectorizer= CountVectorizer(
    tokenizer = lambda x:x , lowercase = False, 
  
    )

train_x= vectorizer.fit_transform(train_texts) # train_texts가 string 이라면 tokenizer 설정 필요 x


print(type(train_x) ) 
train_x

<class 'scipy.sparse.csr.csr_matrix'>


<226572x37134 sparse matrix of type '<class 'numpy.int64'>'
	with 1283074 stored elements in Compressed Sparse Row format>

In [0]:
# 특정 단어에 해당하는 인덱스 알아보기

vocab2int = vectorizer.vocabulary_ #dictionary, {word: index} 형태
print(vocab2int['영화'])

int2vocab = [word for word,index in sorted # 단어를 철자순으로 정렬한 list
             (vocab2int.items(), key=lambda x:x[1])] 


print(int2vocab[24175]) # 위에서 출력된 수 입력



24175
영화


In [0]:
# logistic_regression train
from sklearn.linear_model import LogisticRegression


logistic = LogisticRegression()
logistic.fit(train_x, train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [0]:
# n 번째 리뷰들을 골라 predict 해보기

for idx in [14,100,1000,199999,200006]: # any number
    

text: ['재미없다', '이상', '10자']

predicted class prob: (negative= 0.754, positive= 0.246
predicted class = negative
actual class = negative

--------------------------------------------------

text: ['정말정말', '대단합니다']

predicted class prob: (negative= 0.048, positive= 0.952
predicted class = positive
actual class = positive

--------------------------------------------------

text: ['관람객진짜', '인생영화다', '꼭', '봐야하는', '영화']

predicted class prob: (negative= 0.004, positive= 0.996
predicted class = positive
actual class = positive

--------------------------------------------------

text: ['빵점주고싶다']

predicted class prob: (negative= 0.660, positive= 0.340
predicted class = negative
actual class = negative

--------------------------------------------------

text: ['나도', '최악에', '한표', 'ㅋㅋ']

predicted class prob: (negative= 0.921, positive= 0.079
predicted class = negative
actual class = negative

--------------------------------------------------



In [0]:
#coefficients 살펴보기

# 긍정리뷰로 예측하는데 영향을 미친 단어들 상위 10
coefficients = logistic.coef_.tolist()
sorted_coefficients = sorted(enumerate(coefficients[0]), 
                             key=lambda x:x[1], reverse=True)

for index, coef in sorted_coefficients[:10]:
    print('%s (%.3f)' % (int2vocab[index], coef))

알이즈웰 (4.316)
최고입니다 (3.503)
관람객재밌어요 (3.227)
기대됩니다 (3.199)
10점준다 (3.118)
좋았어요 (3.089)
웰 (2.987)
재미있었습니다 (2.978)
꿀잼 (2.934)
인생영화 (2.934)


In [0]:
# 부정리뷰로 예측하는데 영향을 미친 단어들 상위 10


0점은 (-4.261)
최악 (-4.248)
최악의 (-4.187)
0점이 (-3.892)
1점준다 (-3.749)
돈아깝다 (-3.736)
쓰레기영화 (-3.608)
1점대 (-3.566)
노잼 (-3.559)
최악의영화 (-3.474)


In [0]:
# 5-fold-cross-validation 으로 accuracy, precision, recall 등 구하기

In [0]:
# Train, Test set을 나누어  성능 측정 (accuracy, recall, precision, F1)

In [0]:
# 다른 머신러닝 알고리즘 사용해서 성능 비교 

In [0]:
# from konlpy.tag import Twitter # hannanum etc..
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
# from sklearn.model_selection import cross_val_predict, train_test_split
# from sklearn.metrics import confusion_matrix


# konlpy를 이용한 품사 분석

# 빈도수, 품사 등을 이용한 stop words 처리


